In [28]:
# import json
# import os
# import re
# import urllib.request

# from bs4 import BeautifulSoup
# from requests import get
# from urllib.error import HTTPError
# from urllib.parse import urlencode

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver


In [29]:


# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome('D:/000.work/100_MondayLab/temp/chromedriver_win32/chromedriver')
driver.implicitly_wait(3)

driver.get('https://login.itooza.com/login.htm?qurl64=aHR0cDovL3d3dy5pdG9vemEuY29tLw==')

# PhantomJS의 경우 | 아까 받은 PhantomJS의 위치를 지정해준다.
# driver = webdriver.PhantomJS('D:/000.work/100_MondayLab/temp/phantomjs-2.1.1-windows/bin/phantomjs')

In [30]:
# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('txtUserId').send_keys('jseung21')
driver.find_element_by_name('txtPassword').send_keys('goddns21')

# 로그인 버튼을 눌러주자.
driver.find_element_by_css_selector('div.btn-login>input').click()

In [31]:
# 페이지 이동
code = '035420' # naver
quarter = 'http://www.itooza.com/vclub/y10_page.php?cmp_cd={code}&mode=db&ss=10&sv=1&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1'.format(code=code)
driver.get(quarter)

In [32]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [34]:
# 재무항목
financialColumns=soup.select('div.a01 > table > tbody > tr > th')
financialColumns[:5]

[<th nowrap="" scope="row" style="padding:0px 10px">유동자산</th>,
 <th nowrap="" scope="row" style="font-weight:bold"><span class="expander" style="margin-left: -12px; padding-left: 12px"></span>당좌자산</th>,
 <th nowrap="" scope="row" style="padding: 0px 10px 0px 14px;">현금및현금성자산</th>,
 <th nowrap="" scope="row" style="padding: 0px 10px 0px 14px;">단기금융자산</th>,
 <th nowrap="" scope="row" style="padding:0px 10px">단기금융상품</th>]

In [35]:
# get date
# soup.find("table", {"id":"y10_tb_2"}).find("th")
dates = soup.select('table[id=y10_tb_2] > thead > tr > th')
dates[:5]

[<th class="small t" scope="col" style="display:none"></th>,
 <th class="small" nowrap="" scope="col" style="padding:0px 6px"><span class="en">2018.12</span>월</th>,
 <th class="small" nowrap="" scope="col" style="padding:0px 6px"><span class="en">2018.09</span>월</th>,
 <th class="small" nowrap="" scope="col" style="padding:0px 6px"><span class="en">2018.06</span>월</th>,
 <th class="small" nowrap="" scope="col" style="padding:0px 6px"><span class="en">2018.03</span>월</th>]

In [37]:
# get data
data = soup.select('table[id=y10_tb_2] > tbody > tr > td')
data[:5]

[<td>N/A</td>,
 <td>15,045</td>,
 <td>24,134</td>,
 <td>24,391</td>,
 <td>26,299</td>]

In [39]:
for n in dates:
    print(n.text)


2018.12월
2018.09월
2018.06월
2018.03월
2017.12월
2017.09월
2017.06월
2017.03월
2016.12월
2016.09월
2016.06월
2016.03월
2015.12월
2015.09월
2015.06월
2015.03월
2014.12월
2014.09월
2014.06월
2014.03월
2013.12월
2013.09월
2013.06월
2013.03월
2012.12월
2012.09월
2012.06월
2012.03월
2011.12월
2011.09월
2011.06월
2011.03월
2010.12월
2010.09월
2010.06월
2010.03월
2009.12월
2009.09월
2009.06월
2009.03월
2008.12월
2008.09월
2008.06월
2008.03월
2007.12월
2007.09월
2007.06월
2007.03월
2006.12월
2006.09월
2006.06월
2006.03월
2005.12월
2005.09월
2005.06월
2005.03월
2004.12월
2004.09월
2004.06월
2004.03월
2003.12월
2003.09월
2003.06월
2003.03월
2002.12월
2002.09월
2002.06월
2002.03월
2001.12월
2001.09월
2001.06월
2001.03월
2000.12월
2000.09월
2000.06월
2000.03월
1999.12월
1999.09월
1999.06월
